# 00 — Data Ingestion (Step-by-step)
**Goal:** Walk through using `get_market_data` and `get_options_data` with caching.

---


In [1]:
# 1. Environment setup
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

from datetime import date, timedelta
from oami.config import initialize_environment
from oami.data_layer import get_market_data, get_options_data


api_key = initialize_environment()
print('API key configured' if api_key != 'YOUR_KEY_HERE' else 'Running in offline mode using cache')


✅ Polygon API key detected and loaded.
API key configured


## 2. Choose symbol and date window
We'll work with `SPY` and fetch the last 30 days of data.


In [2]:
symbol = 'SPY'
end_date = date.today()
start_date = end_date - timedelta(days=30)
print(f'Requesting data from {start_date} to {end_date}')


Requesting data from 2025-09-22 to 2025-10-22


## 3. Fetch market data
`get_market_data` reads from the consolidated HDF5 cache at `data/cache/oami_store.h5`. When a requested window is missing, the helper fetches gaps from Polygon and persists them immediately.

In [3]:
market_df = get_market_data(
    symbol=symbol,
    start=start_date.isoformat(),
    end=end_date.isoformat(),
    interval='1D',
    use_cache=True,
)
print('Market rows:', len(market_df))
market_df.head()
market_df.info()
market_df.describe()

Market rows: 21
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  21 non-null     datetime64[ns]
 1   Open       21 non-null     float64       
 2   High       21 non-null     float64       
 3   Low        21 non-null     float64       
 4   Close      21 non-null     float64       
 5   Volume     21 non-null     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 1.1 KB


,Timestamp,Open,High,Low,Close,Volume
count,21,21.000000,21.000000,21.000000,21.000000,2.100000e+01
mean,2025-10-06 11:51:25.714285824,666.160476,668.843629,662.303281,665.571429,7.883018e+07
min,2025-09-22 21:00:00,657.170000,659.405600,652.840000,653.020000,5.462326e+07
25%,2025-09-29 21:00:00,662.930000,665.755000,658.140000,662.230000,6.650194e+07
50%,2025-10-06 21:00:00,666.820000,669.370000,661.860000,665.170000,7.254537e+07
75%,2025-10-13 21:00:00,670.450000,672.675000,667.670000,669.220000,8.628803e+07
max,2025-10-20 21:00:00,673.530000,673.950000,669.981000,673.110000,1.594226e+08
std,NaN,5.164039,4.190386,5.663808,5.189535,2.333328e+07


## 4. Fetch options data
`get_options_data` returns contract metadata derived from the HDF5 cache and backfills any missing expirations or strike slices before responding.

In [4]:
options_df = get_options_data(
    symbol=symbol,
    start_date=start_date.isoformat(),
    end_date=end_date.isoformat(),
    interval='1D',
    use_cache=True,
    look_forward=14,
)
print('Options rows:', len(options_df))
options_df.head()
options_df.info()
options_df.describe()

Options rows: 129
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ticker  129 non-null    object
 1   ohlcv   129 non-null    object
dtypes: object(2)
memory usage: 2.1+ KB


,ticker,ohlcv
count,129,129
unique,129,129
top,O:SPY251103C00630000,o h l c v t...
freq,1,1


In [12]:
options_df.iloc[0][6]

In [ ]:
from oami.utils.cache_manager import H5_PATH
import pandas as pd

with pd.HDFStore(H5_PATH, mode='r') as store:
    print('Cache located at:', H5_PATH)
    print('Available keys:')
    for key in store.keys():
        print(' ', key)


## 5. Inspect the cache keys
Use `pandas.HDFStore` to view the contents of the cache. Keys under `/stocks` contain market bars, while `/options` embed per-contract aggregates.

## 6. Re-using cached data
Running the notebook again with the same window now pulls everything directly from the HDF5 cache—no additional API calls required.